В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'magic_ON_QnSC_100k'
dataname_initial = 'magic'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_100k'

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
idx = [5, 6]

for sigma in [0.01, 0.25, 0]:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/magic_ON_QnSC_100k` folder


In [7]:
normalizers

{'len_num_prev': 10,
 'len_cat_prev': 0,
 'len_target_prev': 1,
 'num_normalizer': QuantileTransformer(n_quantiles=570, output_distribution='normal',
                     random_state=0, subsample=1000000000),
 'cat_normalizer': StandardScaler()}

### Sample + Eval

In [8]:
CONFIG.get_all_args()

{'dataname': 'magic_ON_QnSC_100k',
 'method': None,
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': None,
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 1,
 'real_path': './synthetic/magic_ON_QnSC_100k/initial_real.csv',
 'test_path': './synthetic/magic_ON_QnSC_100k/initial_test.csv',
 'info_path': './data/magic_ON_QnSC_100k/initial_info.json',
 'save_path': None,
 'num_clusters': 25,
 'save_cat': None}

In [9]:
model_save_path_hist

['./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_const_0_01',
 './models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_const_0_25',
 './models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_const_0',
 './models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_mult_5',
 './models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_mult_6']

In [10]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False) # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_const_0_01 tabddpm_ON_QnSC_100k_const_0_01 ./synthetic/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_const_0_01.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([17118, 11])
Sampling time: 93.89070439338684
Sample path changed to ./synthetic/magic_ON_QnSC_100k/initial_tabddpm_ON_QnSC_100k_const_0_01.csv
TABDDPM_ON_QNSC_100K_CONST_0_01: MAGIC_ON_QNSC_100K Dataset
Original Logistic: 0.833
Synthetic Logistic: 0.830
Original Tree: 0.872
Synthetic Tree: 0.872 

TABDDPM_ON_QNSC_100K_CONST_0_01 Accuracy Loss: MAGIC_ON_QNSC_100K
Logistic: 0.318%
Tree: 0.047% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 11/11 [00:00<00:00, 126.18it/s]|
Column Shapes Score: 99.04%

(2/2) Evaluating Column Pair Trends: |██████████| 55/55 [00:00<00:00, 108.57it/s]|
Column Pair Trends Score: 97.69%

Overall 

DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:06<00:00,  5.52it/s]


Saving scores to  eval/mle/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_const_0_01.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_100K_CONST_0_01: 0.949 ± 0.010
magic_ON_QnSC_100k, tabddpm_ON_QnSC_100k_const_0_01: 0.9802873209265949
DCR Score, a value closer to 0.5 is better
magic_ON_QnSC_100k-tabddpm_ON_QnSC_100k_const_0_01, DCR Score = 0.9393620750087627
=========== All Features ===========
Data shape:  (17118, 12)
alpha precision: 0.978210, beta recall: 0.618265
./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_const_0_25 tabddpm_ON_QnSC_100k_const_0_25 ./synthetic/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_const_0_25.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([17118, 11])
Sampling time: 92.34795641899109
Sample path changed to ./synthetic/magic_ON_QnSC_100k/initial_tabddpm_ON_QnSC_100k_const_0_25.csv
TABDDPM_ON_QNSC_100K_CONST_0_25: MAGIC_ON_QNSC_100K Dataset
Original Logistic: 0.833
Synthetic Logistic:

...

Saving scores to  eval/mle/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_mult_5.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_100K_MULT_5: 0.928 ± 0.005
magic_ON_QnSC_100k, tabddpm_ON_QnSC_100k_mult_5: 0.9978806107271246
DCR Score, a value closer to 0.5 is better
magic_ON_QnSC_100k-tabddpm_ON_QnSC_100k_mult_5, DCR Score = 0.9146512443042412
=========== All Features ===========
Data shape:  (17118, 12)
alpha precision: 0.992337, beta recall: 0.493574
./models/tabddpm_ON_QnSC/ckpt/magic/tabddpm_ON_QnSC_100k_mult_6 tabddpm_ON_QnSC_100k_mult_6 ./synthetic/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_mult_6.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([17118, 11])
Sampling time: 92.60251188278198
Sample path changed to ./synthetic/magic_ON_QnSC_100k/initial_tabddpm_ON_QnSC_100k_mult_6.csv
TABDDPM_ON_QNSC_100K_MULT_6: MAGIC_ON_QNSC_100K Dataset
Original Logistic: 0.833
Synthetic Logistic: 0.829
Original Tree: 0.873
Syntheti

DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:06<00:00,  5.23it/s]


Saving scores to  eval/mle/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_mult_6.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_100K_MULT_6: 0.925 ± 0.005
magic_ON_QnSC_100k, tabddpm_ON_QnSC_100k_mult_6: 1.0
DCR Score, a value closer to 0.5 is better
magic_ON_QnSC_100k-tabddpm_ON_QnSC_100k_mult_6, DCR Score = 0.905304357985746
=========== All Features ===========
Data shape:  (17118, 12)
alpha precision: 0.994073, beta recall: 0.493558


In [11]:
final_metrics_table = []
model_short = 'tabddpm_ON_QnSC_100k'
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [12]:
final_metrics_table#.drop(columns=('base_metrics', 0))

\
                  Model                             Type                Data   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_01  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_25  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k     tabddpm_ON_QnSC_100k_const_0  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_5  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_6  magic_ON_QnSC_100k   

       base_metrics                                                  \
  Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0          0.833056           0.830409      0.872102       0.871690   
0          0.833056           0.833471      0.876099       0.864715   
0          0.833056           0.830189      0.871200       0.863766   
0          0.833056           0.832013      0.876000       0.866694   
0          0.833056           0.829370      0.873499       0.856914   

                                                              similarity  ...  \
  Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %  ...   
0                  0.317760              0.047230              99.035040  ...   
0                 -0.049727              1.299388              99.062656  ...   
0                  0.344250              0.853334              98.853944  ...   
0                  0.125218              1.062384              99.140724  ...   
0                  0.442519              1.898683              99.095051  ...   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                            0.964960   
0                                            0.937344   
0                                            1.146056   
0                                            0.859276   
0                                            0.904949   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                0.286866   
0                                                0.257747   
0                                                0.360547   
0                                                0.380000   
0                                                0.290819   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                2.305913   
0                                                1.343938   
0                                                1.063164   
0                                                1.557830   
0                                                1.148806   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                    8.424607   
0                                                    1.043176   
0                                                    1.255094   
0                                                    3.011484   
0                                                    2.129984   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                   0.949132   
0                                   0.884717   
0                                   0.950833   
0                                   0.928089   
0                                   0.924537   

                                                  detection       DCR  \
  ROC - AUC обучения на синтетических данных, std     Score     Score   
0                                        0.010096  0.980287  0.939362   
0                                        0.005091  0.992236  0.906064   
0                                        0.006802  0.976722  0.943919   
0                                    

In [13]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
                  Model                             Type                Data   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_6  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k     tabddpm_ON_QnSC_100k_const_0  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_25  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_5  magic_ON_QnSC_100k   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_01  magic_ON_QnSC_100k   

       base_metrics                                                  \
  Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0            0.8331             0.8294        0.8735         0.8569   
0            0.8331             0.8302        0.8712         0.8638   
0            0.8331             0.8335        0.8761         0.8647   
0            0.8331             0.8320        0.8760         0.8667   
0            0.8331             0.8304        0.8721         0.8717   

                                                              similarity  \
  Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                    0.4425                1.8987                99.0951   
0                    0.3443                0.8533                98.8539   
0                   -0.0497                1.2994                99.0627   
0                    0.1252                1.0624                99.1407   
0                    0.3178                0.0472                99.0350   

                                                          \
  Column Pair Trends Score, % Overall Score (Average), %   
0                     98.8512                    98.9731   
0                     98.9368                    98.8954   
0                     98.6561                    98.8594   
0                     98.4422                    98.7914   
0                     97.6941                    98.3646   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              0.9049   
0                                              1.1461   
0                                              0.9373   
0                                              0.8593   
0                                              0.9650   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  0.2908   
0                                                  0.3605   
0                                                  0.2577   
0                                                  0.3800   
0                                                  0.2869   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                  1.1488   
0                                                  1.0632   
0                                                  1.3439   
0                                                  1.5578   
0                                                  2.3059   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                      2.1300   
0                                                      1.2551   
0                                                      1.0432   
0                                                      3.0115   
0                                                      8.4246   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                     0.9245   
0                                     0.9508   
0                                     0.8847   
0                                     0.9281   
0                                     0.9491   

                

In [14]:
f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv'

'./eval/total/magic_ON_QnSC_100k/tabddpm_ON_QnSC_100k_final_metrcs_table.csv'

In [15]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)